---------------------------------------------------
<h1>TELCO CHURN ANALYSIS</h1>
  <h6> by David Berchelmann -- March 9, 2021 </h6>
  
  ------------------------------------------------

<h3><u>We want to answer a question</u></h3>
          
</br>

<i><font color = red><h4>Why are customers churning?</h4></font></i>
</br>
   - <b>How will we go about this?</b>
        * Acquire the Data
        * Prepare 
        * Explore 
        * Evaluate
        * Create Recommendations Based On Findings </list>

--------------------------

<h5>Let's import some libaries</h5>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



<h3>Time to Acquire the Data</h3>

In [6]:
from acquire import get_telco_data

In [8]:
# Acquire Stage
df = get_telco_data()